In [ ]:
print("Um cuspidor esta nascendo!")

Um cuspidor esta nascendo!


In [1]:
!pip install gradio duckdb pandas --quiet


In [ ]:
#csv operations
import chardet
import pandas as pd
import csv


def get_file_eoptions(arquivo):
    options = {}
    sample = f.read(2048)  # lê uma amostra do arquivo
    sniffer = csv.Sniffer()
    dialect = sniffer.sniff(sample)
    options["delimiter"] = dialect.delimiter
    options["encoding"] = chardet.detect(raw_data)['encoding']
    return options


def carregar_bronze(arquivo):
    if arquivo is None:
        return "Envie um arquivo CSV."
    df = pd.read_csv(arquivo.name,delimiter=op["encoding"],encoding=op["encoding"])
    duckdb.register("bronze", df)
    return df.head(5)

In [5]:
import gradio as gr
import pandas as pd
import csv
import duckdb




with gr.Blocks(title="Arquitetura Medalhão - Data Spitter - CSV - multi Reads") as demo:
    gr.Markdown("#Arquitetura Medalhão (Bronze / Silver / Gold)")
    gr.Markdown("Leia seus dados, configure as operacoes, e maos a obra. Apresentamos suporte a cargas incrementais e de sobrescrita")


    # Camada Bronze
    with gr.Tab("Camada Bronze"):
        gr.Markdown("### Upload dos dados brutos")
        arquivo_bronze = gr.File(label="Selecione um CSV")
        bronze_tabela = gr.DataFrame(label="Prévia dos dados Bronze")
        arquivo_bronze.upload(carregar_bronze, inputs=arquivo_bronze, outputs=bronze_tabela)

    # # Camada Silver
    # with gr.Tab("Camada Silver"):
    #     gr.Markdown("### Limpeza e padronização dos dados")
    #     botao_silver = gr.Button("Transformar Bronze → Silver")
    #     silver_tabela = gr.DataFrame(label="Prévia dos dados Silver")
    #     botao_silver.click(transformar_para_silver, outputs=silver_tabela)

    # # Camada Gold
    # with gr.Tab("Camada Gold"):
    #     gr.Markdown("### Agregações e métricas finais")
    #     botao_gold = gr.Button("Transformar Silver → Gold")
    #     gold_tabela = gr.DataFrame(label="Tabela final Gold")
    #     botao_gold.click(transformar_para_gold, outputs=gold_tabela)

demo.launch(inline=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://972b3197a0e2818b13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
